# 다이소 재고 검색 프로젝트

- 기능 1. 해당 상품명 검색 시 추천순(or 판매량, 리뷰많은순) 상품 5개 선택
- 기능 2. 선택된 상품들에 대해서 리뷰 데이터 수집 (별점, 내용)
- 기능 3. 입력받은 지역 근처 다이소 매장 검색 (3개)
- 기능 4. 선택된 상품과 선택한 매장에 대해 재고 검색

> 입력: 상품명, 지역  
> 출력: 상품 리스트와 리뷰, 재고

다이소몰 URL: https://www.daisomall.co.kr/ds

### 입력 받기 & 필요한 모듈 임포트

In [1]:
import time
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from IPython.display import clear_output 

### 상품 입력해서 상위 5개 항목의 정보 가져오는 함수


In [2]:
def get_products_info(good, browser) :
    browser.get('https://www.daisomall.co.kr/ds/dst/SCR_DST_0015?searchTerm='+good)
    time.sleep(2)
    goods = browser.find_elements(By.CLASS_NAME, "goods-unit")
    
    names = []
    prices = []
    links = []
    
    for good in goods[0:5] :
        names.append(good.find_element(By.CLASS_NAME, "tit").text.replace("BEST","").strip())
        prices.append(good.find_element(By.CLASS_NAME, "value").text)
        links.append(good.find_element(By.TAG_NAME, "a").get_attribute("href"))
        
    data = {"names" : names, "prices" : prices, "links" : links}
    df = pd.DataFrame(data=data)
    
    return df

### 상품의 상세 리뷰 가져오는 함수

In [3]:
def get_review(url, browser) :
    browser.get(url)
    time.sleep(2)
    
    response = browser.page_source
    
    review_page = bs4.BeautifulSoup(response)
    
    one_page_all_reviews = review_page.find_all('div',{'class':'cont'})
    one_page_all_scores = review_page.find_all('span',{'class':'score'})
    
    if not one_page_all_reviews :
        return pd.DataFrame()
    
    else:
        score_list = [score.get_text(strip = True) for score in one_page_all_scores]
        review_list = [review.get_text(strip = True) for review in one_page_all_reviews]
    
    df = pd.DataFrame({'text': review_list, 'score': score_list})
    
    return df


### 해당 지역의 다이소 매장 찾는 함수

In [4]:
def get_store(region, browser):
    search_url = 'https://www.daisomall.co.kr/ms/msg/SCR_MSG_0019'
    browser.get(search_url)

    time.sleep(3) 
    
    # 광고 제거
    btn = browser.find_element(by=By.XPATH, value='//*[@id="btn-close-nday"]')
    btn.click()
    time.sleep(1)
    
    # 매장 검색 박스
    search_box = browser.find_elements(by=By.CLASS_NAME, value='el-input__inner')[1] # index 0: product search box
    search_box.clear()
    search_box.send_keys(region)
    search_box.send_keys(Keys.ENTER)
    time.sleep(3)
    
    # 매장 당 정보 가져오기
    region_boxies = browser.find_elements(by=By.CLASS_NAME, value='store-map')
    store_list = []
    
    for i, region_box in enumerate(region_boxies):
        store = region_box.find_element(by=By.CLASS_NAME, value='tit-h5').text
        location = region_box.find_element(by=By.XPATH, value='//*[@id="pane-2"]/div/div[2]/div[{}]/div[2]/div[2]'.format(i+1)).text
        info = region_box.find_element(by=By.CLASS_NAME, value='info-group')
        opening = info.find_elements(by=By.TAG_NAME, value='span')[0].text.replace('영업 시간 ', '')
        tel = info.find_elements(by=By.TAG_NAME, value='span')[1].text.replace('매장 전화 ', '')
    
        store_list.append({'store':store.split()[0], 'location':location, 'opening':opening, 'tel':tel})
        
        store_df = pd.DataFrame(store_list)
        
    return store_df

### 특정 상품의 특정 매장에서의 재고 검색

In [5]:
def get_stock(product_no, store, browser):
    url = 'https://www.daisomall.co.kr/ms/msg/SCR_MSG_0019'
    browser.get(url)
    time.sleep(3)
    
    # step 1. 매장 상품 찾기 클릭
    btn = browser.find_element(by=By.CSS_SELECTOR, value='#tab-3 > button')
    btn.click()
    time.sleep(3)
    
    # step 2. 특정 상품 검색 및 선택
    search_box = browser.find_element(by=By.CSS_SELECTOR, value='#pane-3 > div > div.form-wrap > div:nth-child(1) > form > div > div > div > div > input')
    search_box.clear()
    search_box.send_keys(product_no)
    search_box.send_keys(Keys.ENTER)
    time.sleep(3)
    
    check_btn = browser.find_element(by=By.CLASS_NAME, value='goods-check')
    check_btn = check_btn.find_element(by=By.TAG_NAME, value='label')
    check_btn.click()
    time.sleep(3)
    
    choice_btn = browser.find_element(by=By.CSS_SELECTOR, value='#__layout > section > div.wrap.scrollArea > div > div:nth-child(3) > div > div.el-dialog__footer > div > button')
    choice_btn.click()
    time.sleep(3)
        
    # step 3. 해당 매장 검색 및 선택
    search_box = browser.find_element(by=By.CSS_SELECTOR, value="#pane-3 > div > div.form-wrap > div:nth-child(2) > form > div > div > div > div > div.el-input.el-input--suffix > input")
    search_box.clear()
    search_box.send_keys(store)
    search_box.send_keys(Keys.ENTER)
    time.sleep(3)
        
    check_btn = browser.find_element(by=By.CLASS_NAME, value='el-radio__input')
    check_btn.click()
    time.sleep(3)
    
    choice_btn = browser.find_element(by=By.CSS_SELECTOR, value='#__layout > section > div.wrap.scrollArea > div > div:nth-child(4) > div > div.el-dialog__footer > div > button')
    choice_btn.click()
    time.sleep(3)
    
    # step 4. 재고 정보 가져오기
    try:                    # 일반적인 유형의 결과 (재고 현황 or 품절 or 단종 or -)
        stock_info = browser.find_element(by=By.CSS_SELECTOR, value='#pane-3 > div > div.form-wrap > div.store-result-area > div > div.store-info > div.info-group').text
    except Exception as e:  # 취급하지 않는 상품의 결과
        stock_info = 'ZONE - 취급하지 않는 상품'
    
    # step 5. 재고 결과 처리
    stock_info = stock_info.split()
    zone = ' '.join(stock_info[:2])
    stock =  ' '.join(stock_info[2:])
    
    if '품절' in stock:
        stock = '품절'
    elif '단종' in stock:
        stock = '단종'
    elif stock == '-':
        stock = '-'
    elif stock == '취급하지 않는 상품':
        pass
    else:
        stock = ' '.join(stock.split()[1:3])
        
        
    return stock, zone

---

### 메인

In [6]:
# 브라우저 옵션 설정 및 브라우저 실행
options = webdriver.EdgeOptions()
options.add_argument('--no-sandbox')                    
options.add_argument('--disable-dev-shm-usage')    

driver = webdriver.Edge(options = options)
driver.maximize_window()

# DataFrame 설정
pd.set_option('display.max_colwidth',500)

In [7]:
# 상품 정보
product = input('상품명을 입력해주세요\n')
df = get_products_info(product, driver)

# 리뷰 정보
review_ls = []
for url in df["links"]:
    review_ls.append(get_review(url,driver))

display(df)

,names,prices,links
0,볼이 빵빵한 친구들 캐릭터 볼펜 세트 3개입,"2,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1048106
1,심슨 실리콘 볼펜 0.5 mm,"1,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1031627
2,볼이 빵빵한 친구들 피규어 회전 볼펜,"2,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1046491
3,미니멜로우 워터스틱 볼펜,"2,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1018974
4,마이멜로디 볼펜 검정 0.5mm,"1,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1036383


In [8]:
# 리뷰 출력
while True :
    answer = input("리뷰를 보고 싶은 상품을 숫자로 선택해주세요(0~4) 보고 싶은 상품이 없을 경우 \"N\"을 입력해주세요")
    clear_output(wait=True)
    if answer == "N" :
        break
    else :
        if review_ls[int(answer)].empty:
            print("리뷰가 없습니다.")
        else :
            display(review_ls[int(answer)])

,text,score
0,귀엽고 뷴홍분홍하니 이뻐요,5점
1,재구매가성비 좋아서 재구매했어요 저렴하게 잘샀어요,5점
2,재구매가성비 좋아서 재구매했어요 저렴하게 잘샀어요,5점
3,별 한개도 아까움 .. 스무스하지못함 ㅠ,1점
4,딸 선물줬어영. 마음에 들어합니다,5점
5,재구매아주좋아요 잘 쓰겠습니다,5점
6,재구매아주좋아요 잘 쓰겠습니다,5점
7,귀여워서 종류별로 다 샀어요 필기감은 보통입니다. 잉크 끊김도 중간중간있음,5점
8,넘 맘에 들어요 0.38이나 젤펜 등 다른 것도 나왔으면 좋겠어요,5점
9,재구매아이들이 너무 좋아합니다,5점


In [9]:
# 매장 정보
region = input("매장을 찾고 싶은 곳의 지역을 입력해주세요.\n")

store_df = get_store(region, driver)
display(store_df)

,store,location,opening,tel
0,세종본점,세종특별자치시 세종로 1219,10:00 ~ 22:00,1522-4400
1,세종나성점,세종특별자치시 나성로 96(나성동),10:00 ~ 22:00,1522-4400
2,세종부강점,세종특별자치시 부강면 부강로 41,10:00 ~ 22:00,044-864-4959
3,세종시청점,세종특별자치시 대평로 71 (대평동),10:00 ~ 22:00,1522-4400
4,세종법원로점,세종특별자치시 소담로 70 (소담동),10:00 ~ 22:00,1522-4400
5,세종첫마을점,세종특별자치시 한누리대로 296 (나성동),10:00 ~ 22:00,1522-4400
6,여주대점,경기 여주시 세종로 274,10:00 ~ 22:00,031-885-9749
7,조치원점,세종특별자치시 조치원읍 조치원4길 7,10:00 ~ 22:00,1522-4400
8,조치원본점,세종특별자치시 조치원읍 충현로 34,10:00 ~ 22:00,1522-4400
9,서울시청광장점,서울특별시 중구 세종대로 93 (태평로2가),09:30 ~ 22:00,1522-4400


In [11]:
# 재고 검색
product_num_list = list(map(int, input("아까 본 상품 중 맘에 드는 상품을 숫자로 골라주세요.(0~4, 중복 선택 시 공백으로 구분)\n").split()))
input_store_list = list(map(int, input("해당 상품을 찾고 싶은 매장의 번호를 입력해주세요.(0~9, 중복 선택 시 공백으로 구분)\n").split()))

for p_no in product_num_list:
    product_prices = df.iloc[p_no].prices
    product_name = df.iloc[p_no].names
    print()
    print(f'{product_name}({product_prices}원)')
    
    for s_no in input_store_list:
        store = store_df.iloc[s_no].store
        stock, location = get_stock(df["links"][p_no][-7:], store, driver)
        
        if stock == '취급하지 않는 상품':
            print(f'>>> 다이소 {store}에서 취급하지 않는 상품입니다.')
        
        print(f'>>> 다이소 {store}의 재고는 {stock}입니다. (위치: {location})')


볼이 빵빵한 친구들 캐릭터 볼펜 세트 3개입(2,000원)
>>> 다이소 세종본점의 재고는 품절입니다. (위치: ZONE 14)
>>> 다이소 세종나성점의 재고는 품절입니다. (위치: ZONE 21)

심슨 실리콘 볼펜 0.5 mm(1,000원)
>>> 다이소 세종본점의 재고는 단종입니다. (위치: ZONE -)
>>> 다이소 세종나성점의 재고는 단종입니다. (위치: ZONE -)

마이멜로디 볼펜 검정 0.5mm(1,000원)
>>> 다이소 세종본점의 재고는 품절입니다. (위치: ZONE 14)
>>> 다이소 세종나성점의 재고는 5개 이하입니다. (위치: ZONE 21)


In [11]:
# 드라이버 종료
driver.quit()